In [0]:
%matplotlib inline

# WGAN
論文：[Wasserstein GAN](https://arxiv.org/abs/1701.07875)

## WGANとは
真のデータ分布とGeneratorによる生成データの分布の距離の計算に，Wasserstein距離を用いるGAN．

## GANの問題点

既存のGANにおける学習はJensen-Shannonダイバージェンス(JSD)を最小化することで，生成データを真のデータに似せていた．

しかし，JSDに基づく学習では，以下のような問題点がある．

- 学習が不安定

    - 真の分布とモデルの分布が重ならない場合では勾配消失問題が発生する
    
    - GeneratorとDiscriminatorとの学習バランスの設定が難しい（主にDiscriminatorが強くなりすぎる）

- モード崩壊が起こる

- 生成データのクオリティ（学習の進み具合）が損失関数の値から判断しずらい 

## WGANの改良点
- 学習が安定しなかった問題を解決

    - Wasserstein距離を用いることにより，真の分布とモデルの分布が重ならない場合でも微分可能に（勾配消失が起こらない）
    
    - 勾配が得られ続けるので，GeneratorとDiscriminatorのシビアな学習バランス設定がいらない

- モード崩壊を解消

- Wasserstein距離 $\approx$ 生成データの品質 なので学習の進み具合がわかりやすい

## Wasserstein距離

先述のように，従来のJSDを用いたGANにおける問題点の１つに「真の分布とモデルの分布が重ならない場合では勾配消失問題が発生する」というものがあった．

GANでの学習の目的は，真の分布$P_{data}$とモデルの分布$P_{g}$との確率分布を一致させることであるが，重ならない場合ではJSDが定義できない．

例えば，確率分布が重ならない極端な例の場合について，JSDを計算する．

<img src="https://i.imgur.com/R6s5JOe.png" width="60%">

$$
\large
\begin{align}
    \tag{1}
    &D_{JS}(P_{data}\| P_g)=\frac{1}{2}D_{KL}\Bigl(P_{data}\|\frac{P_{data}+P_g}{2}\Bigr)+\frac{1}{2}D_{KL}\Bigl(P_g\mid\mid\frac{P_{data}+P_g}{2}\Bigr)\\
    =&
    \begin{cases}
        \frac{1}{2}\Bigl(\sum_{}^{}1\times {\rm log}\bigl(\frac{1}{1/2}\bigr)+\sum_{}^{}1\times {\rm log}\bigl(\frac{1}{1/2}\bigr)\Bigr)\quad(\theta\neq 0)\\
        0\quad(\theta = 0)
    \end{cases}
    \\
    =&
    \begin{cases}
        log(2)\quad(\theta\neq 0)\\
        0\quad(\theta =0)
    \end{cases}
\end{align}
$$

これをグラフにすると以下のようになる．(論文より引用)

真の分布とモデルの分布が重なる$\theta = 0$の部分でJSDが0になるので不連続となっており，勾配消失していることがわかる．

<img src="https://imgur.com/zGuRVOl.png" width="50%">


WGANではこの勾配消失問題を回避するために，以下の式で定義されるWasserstein距離を使用する．

ここでinfは下限のことであり，厳密には異なるらしいが最小値と同じと捉えてもいいだろう．

$$
\large
\begin{align}
    \tag{2}
    W\bigl(P_{data}(x)\| P_g(x)\bigr) &= \underset{\gamma\in\Pi}{\rm inf}\sum_{x,y}^{}\| x-y\|\:\gamma(x,y)\\
    &=\underset{\gamma\in\Pi}{\rm inf}\;\mathbb{E}_{(x,y)\sim\gamma}\| x-y\|
\end{align}
$$
$$
\begin{align}
\cdot\;&\gamma(x,y)\in\Pi(P_{data},P_g)\\
\cdot\;&\displaystyle\sum_{x}^{}\gamma (x,y)=P_{data}(y) , \displaystyle\sum_{y}^{}\gamma(x,y)=P_{data}(x)
\end{align}
$$

このWasserstein距離を先ほど(確率分布が重ならない場合)と同様に計算すると
$$W\bigl(P_{data}\|P_g\bigr)=|\theta |$$

となり，これをグラフにすると以下のようになる．(論文より引用)

<img src="https://imgur.com/C3DK3sB.png" width="50%">

グラフからわかるように，勾配が連続しており，勾配消失を防ぐことができる．

## 損失関数

Wasserstein距離を式(2)に示したが，これは最適輸送問題と同等であり，線形計画法(LP,Linear Programming)を用いることで解くことができる．

しかし，この手法はGANで扱う画像のように次元が大きい場合では，計算量が非現実的になってしまうので双対表現の式を用いる．

双対表現は最小化問題を最大化問題に変形するもので，証明にはFarkasの補題というものを用いるらしい．

双対表現の式は以下で表される．

$$
\large
\begin{align}
    \tag{3}
    W\bigl(P_{data}(x)\| P_g(x)\bigr) &= \frac{1}{K}\underset{\| f\|_{L<K}}{\rm sup}\mathbb{E}_{x\sim P_{data}}\bigl[f(x)\bigr]-\mathbb{E}_{x\sim P_g}\bigl[f(x)\bigr]
\end{align}
$$
$$
\begin{align}
\cdot\;&f:K{\rm -}リプシッツ連続な関数\\
\cdot\;&K:リプシッツ定数
\end{align}
$$

ここでsupは上限を表す．すごくざっくりな理解では，最小化問題→最大化問題へ変化したということ．

また，関数$f$は$K$-リプシッツ連続な関数であることが，元のWasserstein距離の制約条件から必要となる．

さらに，識別器(critic)の重みパラメータをwとし，criticの出力を$f_w$と表記すると，双対表現の式(3)は以下の式で近似できる．

$$
\large
\begin{align}
    \tag{4}
    L_{critic}=W\bigl(P_{data}(x)\| P_g(x)\bigr) &= \frac{1}{K}\underset{w\in W}{\rm max}\;\mathbb{E}_{x\sim P_{data}}\bigl[f(x)\bigr]-\mathbb{E}_{x\sim P_g}\bigl[f(x)\bigr]
\end{align}
$$

WGANでは式(4)の近似式をcriticの損失関数として採用する．

（詳しい内容は[参考リンク](#参考リンク)がおすすめ）

## 実装に使うテクニック

- Discriminatorの出力に活性化関数を使わない

- OptimizerはRMSPropを用いる
  - 学習率を低めに設定

- Discriminatorのパラメータを小さな値でクリップする

- DiscriminatorのパラメータをGeneratorより多く更新する（Unrolled-GANによるもの？）

- **論文で出てくる難しい式は使わない**

# 実装
PyTorch公式実装(DCGAN)を参考にしました（ほぼ同じ）

Google Colabなどでも実行可能ですが，学習にかなり時間がかかるので参考までに...

まずライブラリやパッケージをimportする．

また，Generatorに入力するノイズベクトルのシードを決定する（ここでは再現性のために999に指定）．

In [0]:
from __future__ import print_function
#%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# Set random seed for reproducibility
manualSeed = 999
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

## パラメータ設定
実行のための各パラメータを定義：

* **dataroot** - データセットフォルダへのパス

* **workers** - いくつのコアでデータをロードするか

* **batch_size** - バッチサイズ

* **image_size** - 学習用画像のサイズ（画像のサイズを変更するにはネットワーク構造を要変更）

* **nc** - 入力画像のカラーチャンネル

* **nz** - Generatorに入力するノイズベクトルの次元数

* **ngf** - 伝搬される特徴マップの深さ（Generator用）

* **ndf** - 伝搬される特徴マップの深さ（Discriminator用）

* **num_epochs** - エポック数

* **lr** - 学習率（DCGANでは0.0002だが，WGANでは0.00005という小さい値に設定）

* **clamp-lower** - Discriminatorのパラメータをクリップするときの下限

* **clamp_upper** - 上限

* **n_critic** - Gのパラメータを１回更新する毎に，Discriminatorのパラメータを [n_critic回]更新

* **ngpu** - GPUの使用枚数（0を指定するとCPUで実行）

In [0]:
# Root directory for dataset
dataroot = "/dataset/"

# Number of workers for dataloader
workers = 2

# Batch size during training
batch_size = 128

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 64

# Number of channels in the training images. For color images this is 3
nc = 3

# Size of z latent vector (i.e. size of generator input)
nz = 100

# Size of feature maps in generator
ngf = 64

# Size of feature maps in discriminator
ndf = 64

# Number of training epochs
num_epochs = 20

# Learning rate for optimizers
lr = 0.00005

# Lower for clipping parameter of Critic(Discriminator)
clamp_lower = -0.01
# Upper for clipping parameter of Critic(Discriminator)
clamp_upper = 0.01

# number of D iters per each G iter
n_critic = 5

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1

## データセット
datasetとdataloaderを作成．ここでは[CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html)を用いる．一番下のコードブロックではデータセットの一部を表示している．

In [0]:
# Create the dataset
dataset = dset.CIFAR10(root=dataroot,
                        download=True,
                        train=True,
                        transform=transforms.Compose([
                            transforms.Resize(image_size),
                            transforms.CenterCrop(image_size),
                            transforms.ToTensor(),
                            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))
# Create the dataloader
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         shuffle=True, num_workers=workers)

# Decide which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

# Plot some training images
real_batch = next(iter(dataloader))
plt.figure(figsize=(8,8))
plt.axis("off")
plt.title("Training Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=2, normalize=True).cpu(),(1,2,0)))

## ネットワーク

### Generator
Generatorを定義する．WGANではロスにWasserstein距離を使うだけなので，GeneratorはDCGANのネットワークと同じ．

Generatorは100次元のノイズベクトルから(128batch, 3ch, 64, 64)のテンソルを生成する．

In [0]:
class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 64 x 64
        )

    def forward(self, input):
        return self.main(input)

In [0]:
# Create the generator
netG = Generator(ngpu).to(device)

# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    netG = nn.DataParallel(netG, list(range(ngpu)))

# Print the model
print(netG)

### Discriminator
DCGANではGeneratorからの出力を受け取り，Sigmoid関数によって最終確率を出力していた．

WGANはその必要がない．Discriminatorの出力の平均をWasserstein距離の計算に使うので，最終層の活性化関数は不要．

したがってDCGANのSigmoid関数のみを取り除いた構造になり，いたってシンプル．

In [0]:
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
        )

    def forward(self, input):
        return self.main(input)

In [0]:
# Create the Discriminator
netD = Discriminator(ngpu).to(device)

# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    netD = nn.DataParallel(netD, list(range(ngpu)))

# Print the model
print(netD)

## Optimizer

最適化アルゴリズムにはRMSPropを採用している．Adamでは学習が不安定になることがあり，RMSPropを使うと改善されたと書かれている：
>We therefore switched to RMSProp which is known to perform well even on very nonstationary problems.

また，学習率は0.00005という低い学習率を設定する．

In [0]:
# Create batch of latent vectors that we will use to visualize
#  the progression of the generator
fixed_noise = torch.randn(64, nz, 1, 1, device=device)

# Setup Adam optimizers for both G and D
optimizerD = optim.RMSprop(netD.parameters(), lr=lr)
optimizerG = optim.RMSprop(netG.parameters(), lr=lr)

## 学習
ロス関数の実装には論文に記載されている式は一切使わない．

本物データに対するDiscriminatorの出力の平均を$f_w(x)$，Generatorによる偽データに対するDの出力の平均を$f_w(\hat{x})$とする．

また，Discriminatorのパラメータを$w$，Generatorのパラメータを$\theta$とする．

学習のステップは以下の通り：

1. $w$を$f_w(x) - f_w(\hat{x})$で更新

2. $w$を$[-c,c]$の範囲でクリップ

3. 1と2を$n_{critic}$回繰り返す

4. $\theta$を$f_w(\hat{x})$で更新

$f_w(x) - f_w(\hat{x})$がWasserstein距離を表している．

Discriminatorは本物のデータに対し大きな値を出力し、偽のデータに対して小さな値を出力する必要がある．したがって，$f_w(x) - f_w(\hat{x})$を最大化する．

$w$を更新する毎に$[-c,c]$でクリップするのは，リプシッツ制約を保つためである．

Generatorは$f_w(\hat{x})$を最大化することで，Wasserstein距離($f_w(x) - f_w(\hat{x})$)を小さくする．(つまり真の分布とモデルの分布を近づける)

1と4のステップでは目的関数を最大化すると記述したが，**PyTorchではロス関数として最小化されるので，目的関数にマイナスをかける．**

したがって，GeneratorとDiscriminatorのロス関数は以下のようになる：
- D： $-f_w(x) + f_w(\hat{x})$
- G： $-f_w(\hat{x})$

コードは以下：

In [0]:
# Training Loop

# Lists to keep track of progress
img_list = []
G_losses = []
D_losses = []
iters = 0

print("Starting Training Loop...")
# For each epoch
for epoch in range(num_epochs):
    # For each batch in the dataloader
    for i, data in enumerate(dataloader, 0):
        
        #############################
        # 3. Repeat [n_critic] times
        #############################
        for n in range(n_critic):

            ############################
            # 1. Update D network
            ###########################
            ## Train with all-real batch
            netD.zero_grad()
            # Format batch
            real_cpu = data[0].to(device)
            b_size = real_cpu.size(0)
            # Forward pass real batch through D
            output = netD(real_cpu).view(-1)
            # Calculate loss on all-real batch
            errD_real = torch.mean(output)
            # Calculate gradients for D in backward pass
            D_x = output.mean().item()

            ## Train with all-fake batch
            # Generate batch of latent vectors
            noise = torch.randn(b_size, nz, 1, 1, device=device)
            # Generate fake image batch with G
            fake = netG(noise)
            # Classify all fake batch with D
            output = netD(fake.detach()).view(-1)
            # Calculate D's loss on the all-fake batch
            errD_fake = torch.mean(output)
            # Calculate the gradients for this batch
            D_G_z1 = output.mean().item()
            # Add the gradients from the all-real and all-fake batches
            errD = - errD_real + errD_fake
            errD.backward()
            # Update D
            optimizerD.step()

            ##################################
            # 2. Clip weights of discriminator
            ##################################
            for p in netD.parameters():
                p.data.clamp_(clamp_lower, clamp_upper)

        ############################
        # 4. Update G network
        ###########################
        netG.zero_grad()
        # Generate batch of latent vectors
        noise = torch.randn(b_size, nz, 1, 1, device=device)
        # Generate fake image batch with G
        fake = netG(noise)
        # Since we just updated D, perform another forward pass of all-fake batch through D
        output = netD(fake).view(-1)
        # Calculate G's loss based on this output
        errG = - torch.mean(output)
        # Calculate gradients for G
        errG.backward()
        D_G_z2 = output.mean().item()
        # Update G
        optimizerG.step()
        
        # Output training stats
        if i % 50 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch+1, num_epochs, i, len(dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))
        
        # Save Losses for plotting later
        G_losses.append(errG.item())
        D_losses.append(errD.item())
        
        # Check how the generator is doing by saving G's output on fixed_noise
        if (iters % 500 == 0) or ((epoch == num_epochs-1) and (i == len(dataloader)-1)):
            with torch.no_grad():
                fake = netG(fixed_noise).detach().cpu()
            img_list.append(vutils.make_grid(fake, padding=2, normalize=True))
            
        iters += 1

## 結果の表示

※学習に時間がかかる．

In [0]:
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(G_losses,label="G")
plt.plot(D_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [0]:
#%%capture
fig = plt.figure(figsize=(8,8))
plt.axis("off")
ims = [[plt.imshow(np.transpose(i,(1,2,0)), animated=True)] for i in img_list]
ani = animation.ArtistAnimation(fig, ims, interval=1000, repeat_delay=1000, blit=True)

HTML(ani.to_jshtml())

In [0]:
# Grab a batch of real images from the dataloader
real_batch = next(iter(dataloader))

# Plot the real images
plt.figure(figsize=(15,15))
plt.subplot(1,2,1)
plt.axis("off")
plt.title("Real Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=5, normalize=True).cpu(),(1,2,0)))

# Plot the fake images from the last epoch
plt.subplot(1,2,2)
plt.axis("off")
plt.title("Fake Images")
plt.imshow(np.transpose(img_list[-1],(1,2,0)))
plt.show()

# ポケモン生成してみた

## 生成結果

左が通常のGANの目的関数，右がWasserstein距離を用いたWGAN．ネットワークはどちらもDCGANのものを使用している．

DCGANの方が早く収束するようだが，学習が不安定でmode collapseを起こしている．

それに対しWGANは学習に時間を要するが安定しており，より鮮明に生成された．

|DCGAN|WGAN|
|:---:|:---:|
|<img src="https://i.imgur.com/7oHacMp.gif" width=80%>|<img src="https://i.imgur.com/S9cHDtV.gif" width=80%>|


## 学習曲線

WGANのDiscriminatorのロス(Wasserstein距離) $\approx$ 生成画像のクオリティー なので学習の進捗がわかりやすい．

DCGANは学習が不安定になっており，Discriminatorが強くなってしまった．

**※Discriminatorのロスが増加しているように見えるが，ロス自体がWasserstein距離を表しているので，0に向かうように収束する．**
- Discriminator

|DCGAN|WGAN|
|:---:|:---:|
|<img src="https://i.imgur.com/cdS6w4T.png">|<img src="https://i.imgur.com/6yhzsH9.png">|

- Generator

|DCGAN|WGAN|
|:---:|:---:|
|<img src="https://i.imgur.com/3uzNZgw.png">|<img src="https://i.imgur.com/ZJN9iiB.png">|

# 気づいたこと

- 学習回数（エポック数）は多めに設定

  - 学習率を低く設定している，つまり学習が遅いのでエポック数は多めに設定

- 通常のGANより時間がかかる

- ロス(Wasserstein距離)の推移は見た方がいい

- 論文ではDiscriminatorに正規化は不要と書かれているが，ポケモン生成ではBatch Normalizationをなくすと結果が悪化した(データセットによるのかも？)

# 参考リンク

- 理論
  - [GANからWasserstein GANへ](https://daiki-yosky.hatenablog.com/entry/2019/04/24/GAN%E3%81%8B%E3%82%89Wasserstein_GAN%E3%81%B8)
  
  - [Wasserstein GAN [arXiv:1701.07875] ご注文は機械学習ですか？](http://musyoku.github.io/2017/02/06/Wasserstein-GAN/)

  - [Wasserstein GAN と Kantorovich-Rubinstein 双対性](https://qiita.com/mittyantest/items/0fdc9ce7624dbd2ee134)

  - [今さら聞けないGAN（4） WGAN](https://qiita.com/triwave33/items/5c95db572b0e4d0df4f0)

  - [情報工学_機械学習_生成モデル.md](https://github.com/Yagami360/My_NoteBook/blob/master/%E6%83%85%E5%A0%B1%E5%B7%A5%E5%AD%A6/%E6%83%85%E5%A0%B1%E5%B7%A5%E5%AD%A6_%E6%A9%9F%E6%A2%B0%E5%AD%A6%E7%BF%92_%E7%94%9F%E6%88%90%E3%83%A2%E3%83%87%E3%83%AB.md#WGAN)

- 実装

   - [論文著者のリポジトリ](https://github.com/martinarjovsky/WassersteinGAN)

   - [GAN_WGAN_PyTorch](https://github.com/Yagami360/MachineLearning_Exercises_Python_PyTorch/tree/master/GAN_WGAN_PyTorch)